In [ ]:
"""
Topic: 实现台式眼底图像数据的标准化表格输出，实现数据的结构化。
@ Author: Yale_yang 
Time: Dec. 9th 2019
"""

In [1]:
import os 
import re
import shutil
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
print('import succed!')
# 设置显示效果
pd.set_option('display.max_columns',None)   # 显示全部列数
pd.set_option('display.max_rows',None)    # 显示全部行
pd.set_option('expand_frame_repr',False)  # 禁止自动换行

import succed!


In [2]:
input_path = r'/Users/edz/Documents/My-work/Daily_work/table_standardization/Taishi_standardization'

df = pd.DataFrame()
for maindir, subdir, file_name_list in os.walk(input_path):
    print(file_name_list)  # 查看文件列表
    for filename in file_name_list:
        if filename.endswith('.xlsx'):  # 判断文件类型
            input_file_path = os.path.join(input_path,filename)   # 拼接文件的绝对路径
            df_i = pd.read_excel(input_file_path,sep='\t',index_col=None)
            print('{} items were added which came from {}!'.format(df_i.shape[0],filename))
            df = df.append(df_i,ignore_index=True)  # 拼接多个表，同时重置索引index
            
print(df.info())

['.DS_Store', 'new-ai-355.xlsx']
2878 items were added which came from new-ai-355.xlsx!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2878 entries, 0 to 2877
Data columns (total 34 columns):
检查ID      2878 non-null int64
bucket    2878 non-null object
图片路径      2878 non-null object
病人ID      2878 non-null int64
出生日期      2878 non-null object
送阅时间      2878 non-null object
性别        2878 non-null int64
地区        0 non-null float64
新生胎龄      2878 non-null int64
矫正胎龄      2878 non-null int64
病例类型      2878 non-null int64
病例医院      2878 non-null object
阅片医院      2878 non-null object
病例医生id    2878 non-null int64
病例医生      2878 non-null object
阅片医生id    2878 non-null int64
阅片医生      2858 non-null object
病史        2870 non-null object
全身病       10 non-null object
病例诊断      0 non-null float64
左眼视力      2855 non-null object
右眼视力      2855 non-null object
左眼眼压      1294 non-null object
右眼眼压      1291 non-null object
左眼矫正视力    377 non-null object
右眼矫正视力    396 non-null object
图像描述      2843 

In [7]:
# df['质控阅片诊断'].value_counts()

双眼高血压性视网膜病变右眼陈旧性视网膜脉络膜病变    3
Name: 质控阅片诊断, dtype: int64

In [ ]:
df.head(2)

In [3]:
# 删除无效列
df.drop(columns=['地区','全身病','病例医生id','病例医生','阅片医生id','阅片医生','病例诊断','左眼矫正视力',
                 '右眼矫正视力','新生胎龄','矫正胎龄'],inplace=True)

In [4]:
column_add_list = ['色素变性','白内障','脉络膜病变','脉络膜萎缩','屈光不正','虹膜缺损','高度近视','弱视','黄斑病变','黄斑前膜',
                   '黄斑裂孔','黄斑水肿','眼睑淤血','屈光间质混浊','动脉硬化','玻璃体混浊','玻璃体后脱离','玻璃体积血','视网膜脱离',
                   '青光眼','结膜下出血','视网膜前出血','营养不良','视神经萎缩','无脉络膜症','RPE病变','高眼压','玻璃膜疣',
                   '翼状胬肉','病理性近视','后巩膜葡萄肿','眼底激光斑','先天性大视杯','血管堵塞','分支静脉阻塞','视网膜病变',
                   '视网膜劈裂','角膜病变','角膜斑翳','老视']

In [ ]:
df.columns

In [ ]:
df.head()

In [5]:
# 插入指定列——阅片诊断
column_i = df.columns.get_loc('阅片诊断')+1  # 获取指定列名的索引位置
for i in range(len(column_add_list)):
    df.insert(column_i,column_add_list[i],None)   # 指定位置插入指定列
    column_i += 1

print(df.columns)

Index(['检查ID', 'bucket', '图片路径', '病人ID', '出生日期', '送阅时间', '性别', '病例类型', '病例医院',
       '阅片医院', '病史', '左眼视力', '右眼视力', '左眼眼压', '右眼眼压', '图像描述', '阅片诊断', '色素变性',
       '白内障', '脉络膜病变', '脉络膜萎缩', '屈光不正', '虹膜缺损', '高度近视', '弱视', '黄斑病变', '黄斑前膜',
       '黄斑裂孔', '黄斑水肿', '眼睑淤血', '屈光间质混浊', '动脉硬化', '玻璃体混浊', '玻璃体后脱离', '玻璃体积血',
       '视网膜脱离', '青光眼', '结膜下出血', '视网膜前出血', '营养不良', '视神经萎缩', '无脉络膜症', 'RPE病变',
       '高眼压', '玻璃膜疣', '翼状胬肉', '病理性近视', '后巩膜葡萄肿', '眼底激光斑', '先天性大视杯', '血管堵塞',
       '分支静脉阻塞', '视网膜病变', '视网膜劈裂', '角膜病变', '角膜斑翳', '老视', '质控影像描述', '质控阅片诊断',
       '病例检查', '处理', '处方', '主诉'],
      dtype='object')


In [6]:
possibility_list = ['？','待查','可能性大','待除外']

In [12]:
import difflib
import string
for i in range(df.shape[0]):
    if pd.isna(df.at[i,'质控阅片诊断']):
        str_i = df.at[i,'阅片诊断']
    else:
        str_i = df.at[i,'质控阅片诊断']
    str_tempt = re.split(r'双眼|右|左|；|及|，|、', str_i)
    str_i_list = list(filter(None,str_tempt))    # list(filter(None,str)) :去掉re.split()产生的”空字符“项
#     print(str_i_list)
    for j in range(len(column_add_list)):
        for k in range(len(str_i_list)):
            if  (str.find(str_i_list[k],column_add_list[j])!=-1) and (any( key in str_i_list[k] for key in possibility_list )):
                df.at[i,column_add_list[j]] = -1   # -1表示”不确定“
#                 print(str_i_list[k])
            elif (str.find(str_i_list[k],column_add_list[j])!=-1) and (all(key not in str_i_list[k] for key in possibility_list)):
                df.at[i,column_add_list[j]] = 1   # 1表示”患该病“
#                 print(column_add_list[j])
            else:
                pass

In [ ]:
import difflib
import string
for i in range(df.shape[0]):
    str_i = df.at[i,'阅片诊断']
    str_tempt = re.split(r'双眼|右|左|；|及|，|、', str_i)
    str_i_list = list(filter(None,str_tempt))    # list(filter(None,str)) :去掉re.split()产生的”空字符“项
#     print(str_i_list)
    for j in range(len(column_add_list)):
        for k in range(len(str_i_list)):
            if  (str.find(str_i_list[k],column_add_list[j])!=-1) and (any( key in str_i_list[k] for key in possibility_list )):
                df.at[i,column_add_list[j]] = -1   # -1表示”不确定“
#                 print(str_i_list[k])
            elif (str.find(str_i_list[k],column_add_list[j])!=-1) and (all(key not in str_i_list[k] for key in possibility_list)):
                df.at[i,column_add_list[j]] = 1   # 1表示”患该病“
#                 print(column_add_list[j])
            else:
                pass

In [13]:
df['屈光不正'].value_counts()

 1    187
-1     73
Name: 屈光不正, dtype: int64

In [25]:
# df[df['质控阅片诊断'].notnull()]  # df.columns.get_loc('阅片诊断')——19

,检查ID,bucket,图片路径,病人ID,出生日期,送阅时间,性别,病例类型,病例医院,阅片医院,病史,左眼视力,右眼视力,左眼眼压,右眼眼压,图像描述,阅片诊断,色素变性,白内障,脉络膜病变,脉络膜萎缩,屈光不正,虹膜缺损,高度近视,弱视,黄斑病变,黄斑前膜,黄斑裂孔,黄斑水肿,眼睑淤血,屈光间质混浊,动脉硬化,玻璃体混浊,玻璃体后脱离,玻璃体积血,视网膜脱离,青光眼,结膜下出血,视网膜前出血,营养不良,视神经萎缩,无脉络膜症,RPE病变,高眼压,玻璃膜疣,翼状胬肉,病理性近视,后巩膜葡萄肿,眼底激光斑,先天性大视杯,血管堵塞,分支静脉阻塞,视网膜病变,视网膜劈裂,角膜病变,角膜斑翳,老视,质控影像描述,质控阅片诊断,病例检查,处理,处方,主诉
2789,100968215,zhizhen-imgcenter,827cb28bd8a34a6f82bc6c5db35e181f.jpg,10301699,1950-12-18,1970-01-01,1,2,北京市朝阳区和平街社区卫生服务中心,至真阅片中心,无病史；,5.0(1.0),10cm手动,NaN,NaN,双眼视盘边清色正，右眼黄斑区色素紊乱及瘢痕，周边视网膜可见骨细胞样色素，静脉血管迂曲扩张，动...,右眼视网膜色素变性右眼黄斑病变,None,None,1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,None,None,None,None,双眼视盘表面血管迂曲，部分出血，视网膜静脉血管迂曲扩张，动脉血管细，动静脉交叉症明显。右后极...,双眼高血压性视网膜病变右眼陈旧性视网膜脉络膜病变,NaN,NaN,NaN,NaN
2790,100968216,zhizhen-imgcenter,20fd7c4dbcadd9132c3f8c3036998ed0.jpg,10301699,1950-12-18,1970-01-01,1,2,北京市朝阳区和平街社区卫生服务中心,至真阅片中心,无病史；,5.0(1.0),10cm手动,NaN,NaN,双眼视盘边清色正，右眼黄斑区色素紊乱及瘢痕，周边视网膜可见骨细胞样色素，静脉血管迂曲扩张，动...,右眼视网膜色素变性右眼黄斑病变,None,None,1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,None,None,None,None,双眼视盘表面血管迂曲，部分出血，视网膜静脉血管迂曲扩张，动脉血管细，动静脉交叉症明显。右后极...,双眼高血压性视网膜病变右眼陈旧性视网膜脉络膜病变,NaN,NaN,NaN,NaN
2791,100968217,zhizhen-imgcenter,cdb99a7ef972d0869f2a96ffd1518ff6.jpg,10301699,1950-12-18,1970-01-01,1,2,北京市朝阳区和平街社区卫生服务中心,至真阅片中心,无病史；,5.0(1.0),10cm手动,NaN,NaN,双眼视盘边清色正，右眼黄斑区色素紊乱及瘢痕，周边视网膜可见骨细胞样色素，静脉血管迂曲扩张，动...,右眼视网膜色素变性右眼黄斑病变,None,None,1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,None,None,None,None,双眼视盘表面血管迂曲，部分出血，视网膜静脉血管迂曲扩张，动脉血管细，动静脉交叉症明显。右后极...,双眼高血压性视网膜病变右眼陈旧性视网膜脉络膜病变,NaN,NaN,NaN,NaN


In [ ]:
df['病史'].value_counts()

In [26]:
desease_column_list = ['无病史','高血压','血脂高','血糖高','糖尿病','低血压','心脏病','冠心病','脑梗','脑血管硬化','肾结石',
                       '脂肪肝','心肌缺血','妊娠','脑卒','视物不清','视物变形','胃病','肾病','视神经炎', '色觉异常','虹睫炎',
                       '结膜淋巴滤泡','眼底出血','近视','斜视','先天性色素变性','夜盲症','智力障碍',
                       '脑瘤术后','白内障术后','甲亢术后','脑出血术后','青光眼术后','视网膜复位手术']

In [27]:
# 插入指定列——病史
column_i = df.columns.get_loc('病史')+1  # 获取指定列名的索引位置
for i in range(len(desease_column_list)):
    df.insert(column_i,desease_column_list[i],None)   # 指定位置插入指定列
    column_i += 1

print(df.columns)

Index(['检查ID', 'bucket', '图片路径', '病人ID', '出生日期', '送阅时间', '性别', '病例类型', '病例医院',
       '阅片医院', '病史', '无病史', '高血压', '血脂高', '血糖高', '糖尿病', '低血压', '心脏病', '冠心病',
       '脑梗', '脑血管硬化', '肾结石', '脂肪肝', '心肌缺血', '妊娠', '脑卒', '视物不清', '视物变形', '胃病',
       '肾病', '视神经炎', '色觉异常', '虹睫炎', '结膜淋巴滤泡', '眼底出血', '近视', '斜视', '先天性色素变性',
       '夜盲症', '智力障碍', '脑瘤术后', '白内障术后', '甲亢术后', '脑出血术后', '青光眼术后', '视网膜复位手术',
       '左眼视力', '右眼视力', '左眼眼压', '右眼眼压', '图像描述', '阅片诊断', '色素变性', '白内障', '脉络膜病变',
       '脉络膜萎缩', '屈光不正', '虹膜缺损', '高度近视', '弱视', '黄斑病变', '黄斑前膜', '黄斑裂孔', '黄斑水肿',
       '眼睑淤血', '屈光间质混浊', '动脉硬化', '玻璃体混浊', '玻璃体后脱离', '玻璃体积血', '视网膜脱离', '青光眼',
       '结膜下出血', '视网膜前出血', '营养不良', '视神经萎缩', '无脉络膜症', 'RPE病变', '高眼压', '玻璃膜疣',
       '翼状胬肉', '病理性近视', '后巩膜葡萄肿', '眼底激光斑', '先天性大视杯', '血管堵塞', '分支静脉阻塞', '视网膜病变',
       '视网膜劈裂', '角膜病变', '角膜斑翳', '老视', '质控影像描述', '质控阅片诊断', '病例检查', '处理', '处方',
       '主诉'],
      dtype='object')


In [29]:
total_add_list = desease_column_list+column_add_list

for i in range(df.shape[0]):
    for j in range(len(total_add_list)):
        if str(df.at[i,'病史']).find(total_add_list[j])!=-1:
#             print(df.at[i,'病史'])
#             print(total_add_list[j])
            df.at[i,total_add_list[j]] = 1   # 确定项填充”1“
        else:
            pass    

In [30]:
# 指定列的Null值项填充“0”
for i in range(len(total_add_list)):
    df[total_add_list[i]].fillna(0,inplace=True)

In [31]:
df.head()

,检查ID,bucket,图片路径,病人ID,出生日期,送阅时间,性别,病例类型,病例医院,阅片医院,病史,无病史,高血压,血脂高,血糖高,糖尿病,低血压,心脏病,冠心病,脑梗,脑血管硬化,肾结石,脂肪肝,心肌缺血,妊娠,脑卒,视物不清,视物变形,胃病,肾病,视神经炎,色觉异常,虹睫炎,结膜淋巴滤泡,眼底出血,近视,斜视,先天性色素变性,夜盲症,智力障碍,脑瘤术后,白内障术后,甲亢术后,脑出血术后,青光眼术后,视网膜复位手术,左眼视力,右眼视力,左眼眼压,右眼眼压,图像描述,阅片诊断,色素变性,白内障,脉络膜病变,脉络膜萎缩,屈光不正,虹膜缺损,高度近视,弱视,黄斑病变,黄斑前膜,黄斑裂孔,黄斑水肿,眼睑淤血,屈光间质混浊,动脉硬化,玻璃体混浊,玻璃体后脱离,玻璃体积血,视网膜脱离,青光眼,结膜下出血,视网膜前出血,营养不良,视神经萎缩,无脉络膜症,RPE病变,高眼压,玻璃膜疣,翼状胬肉,病理性近视,后巩膜葡萄肿,眼底激光斑,先天性大视杯,血管堵塞,分支静脉阻塞,视网膜病变,视网膜劈裂,角膜病变,角膜斑翳,老视,质控影像描述,质控阅片诊断,病例检查,处理,处方,主诉
0,7015385,zhizhen-tms,eznkSljRMFaAXOR4AALlOPRJL8I290.jpg,2365064,1938-10-07,1970-01-01,2,8,内蒙古巴彦淖尔市杭锦后旗医院,北京同仁眼科远程会诊中心,心脏病3年；,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.1,0.1,NaN,NaN,双视盘边清色可，视网膜动静脉交叉压迫证，网膜散在大量结晶样病灶,双眼结晶样视网膜色素变性？,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,双眼视物模糊1年有余
1,7015386,zhizhen-tms,eznkSljRMFaAVf0pAALeuYEZti0235.jpg,2365064,1938-10-07,1970-01-01,2,8,内蒙古巴彦淖尔市杭锦后旗医院,北京同仁眼科远程会诊中心,心脏病3年；,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.1,0.1,NaN,NaN,双视盘边清色可，视网膜动静脉交叉压迫证，网膜散在大量结晶样病灶,双眼结晶样视网膜色素变性？,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,双眼视物模糊1年有余
2,7015387,zhizhen-tms,eznkSljRMFaADDaRAAL-DaEhAuM359.jpg,2365064,1938-10-07,1970-01-01,2,8,内蒙古巴彦淖尔市杭锦后旗医院,北京同仁眼科远程会诊中心,心脏病3年；,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.1,0.1,NaN,NaN,双视盘边清色可，视网膜动静脉交叉压迫证，网膜散在大量结晶样病灶,双眼结晶样视网膜色素变性？,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,双眼视物模糊1年有余
3,7501602,zhizhen-tms,12-10000220120423-20120423151634296_100002_800...,2516552,1973-10-10,1970-01-01,2,8,河南省舞钢市人民医院,北京同仁眼科远程会诊中心,无病史；,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.2,0.006,25,19,双眼底网膜血管细，大片色素沉积，黄斑区色素紊乱,双眼视网膜色素变性,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,左眼视物不清10天
4,7501603,zhizhen-tms,12-10000220120423-20120423151634296_100002_800...,2516552,1973-10-10,1970-01-01,2,8,河南省舞钢市人民医院,北京同仁眼科远程会诊中心,无病史；,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.2,0.006,25,19,双眼底网膜血管细，大片色素沉积，黄斑区色素紊乱,双眼视网膜色素变性,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,左眼视物不清10天


In [32]:
df['屈光不正'].value_counts()

 0    2618
 1     187
-1      73
Name: 屈光不正, dtype: int64

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2878 entries, 0 to 2877
Data columns (total 98 columns):
检查ID       2878 non-null int64
bucket     2878 non-null object
图片路径       2878 non-null object
病人ID       2878 non-null int64
出生日期       2878 non-null object
送阅时间       2878 non-null object
性别         2878 non-null int64
病例类型       2878 non-null int64
病例医院       2878 non-null object
阅片医院       2878 non-null object
病史         2870 non-null object
无病史        2878 non-null int64
高血压        2878 non-null int64
血脂高        2878 non-null int64
血糖高        2878 non-null int64
糖尿病        2878 non-null int64
低血压        2878 non-null int64
心脏病        2878 non-null int64
冠心病        2878 non-null int64
脑梗         2878 non-null int64
脑血管硬化      2878 non-null int64
肾结石        2878 non-null int64
脂肪肝        2878 non-null int64
心肌缺血       2878 non-null int64
妊娠         2878 non-null int64
脑卒         2878 non-null int64
视物不清       2878 non-null int64
视物变形       2878 non-null int64
胃病         2878 no

In [34]:
df['病例检查'].value_counts()

黄斑变性/双                                                                                                                           34
视网膜色素变性/双                                                                                                                        13
右眼后囊下性白内障                                                                                                                        12
双眼外眼未见明显异常，右眼晶状体后囊轻度混浊，左眼晶状体密度增高，双眼底：视乳头边界清，色淡红，后极部网膜呈金箔杨反光，周边网膜可见骨细胞样改变，黄斑中心凹反光消失。OCT提示：双眼黄斑前膜，左眼黄斑板层裂孔                         10
双眼视物色素沉着，考虑：双眼视网膜色素变性？                                                                                                            9
屈光不正；视网膜色素变性                                                                                                                      8
双眼白内障术后                                                                                                                           8
双眼视网膜色素变性                                                                   

In [ ]:
df['主诉'].value_counts()

In [35]:
# 删除无效列
df.drop(columns=['质控影像描述','质控阅片诊断'],inplace=True)

In [36]:
# 数据导出
output_path = r'/Users/edz/Documents/My-work/Daily_work/table_standardization/Result_last'
output_file = r'/Taishi_Table_standardization.xlsx'
writer = pd.ExcelWriter(output_path + output_file)
df.to_excel(writer,sheet_name='Taishi_Table',index=None)
writer.save()

In [52]:
df.iloc[5:55,51:92]

,阅片诊断,色素变性,白内障,脉络膜病变,脉络膜萎缩,屈光不正,虹膜缺损,高度近视,弱视,黄斑病变,黄斑前膜,黄斑裂孔,黄斑水肿,眼睑淤血,屈光间质混浊,动脉硬化,玻璃体混浊,玻璃体后脱离,玻璃体积血,视网膜脱离,青光眼,结膜下出血,视网膜前出血,营养不良,视神经萎缩,无脉络膜症,RPE病变,高眼压,玻璃膜疣,翼状胬肉,病理性近视,后巩膜葡萄肿,眼底激光斑,先天性大视杯,血管堵塞,分支静脉阻塞,视网膜病变,视网膜劈裂,角膜病变,角膜斑翳,老视
5,双眼视网膜色素变性,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,双眼视网膜色素变性,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,双眼视网膜色素变性,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,双眼视网膜色素变性,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,双眼原发性视网膜色素变性,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,双眼原发性视网膜色素变性,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
11,双眼原发性视网膜色素变性,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,双眼原发性视网膜色素变性,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
13,双眼原发性视网膜色素变性,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,双眼原发性视网膜色素变性,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [55]:
df.shape[1]

96

In [ ]:
df['心脏病'].value_counts()

In [ ]:
df['病史'].isnull().sum()

In [ ]:
df.loc[df['病史'].str.contains('心脏病'),'病史']

In [ ]:
df[df['质控阅片诊断'].notnull()][['阅片诊断','质控阅片诊断']]

In [ ]:
# df['图像描述'].value_counts()

In [ ]:
# df['病史'].value_counts()

In [ ]:
# df['病例检查'].value_counts()

In [ ]:
# df['主诉'].value_counts()

In [ ]:
# df['阅片诊断'].value_counts()

In [ ]:
# df.insert(column_i,'阅片诊断_test',df['阅片诊断'])

In [ ]:
# a = '双眼原发性视网膜色素变性双眼并发性白内障 '
# b = re.split(r'双眼',a)
# print(b)
# print(list(filter(None, b)))

# B列表模糊匹配A列表
a = ['123','666','355']
b = ['2','5']
for i in range(len(b)):
    for j in range(len(a)):
        if a[j].find(b[i]) == -1:
            continue
        print(a[j])
        
        
        
a = ['视网膜色素变性', '并发性白内障']
for i in range(len(column_add_list)):
    for j in range(len(a)):
        if a[j].find(column_add_list[i]) == -1:
            continue
        print(a[j])

In [ ]:
import difflib
import string
for i in range(df.shape[0]):
    str_i = df.at[i,'阅片诊断']
    str_tempt = re.split(r'双眼|右|左|；|及|，|、', str_i)
    str_i_list = list(filter(None,str_tempt))    # list(filter(None,str)) :去掉re.split()产生的”空字符“项
#     print(str_i_list)
    for j in range(len(column_add_list)):
        for k in range(len(str_i_list)):
            if  (str.find(str_i_list[k],column_add_list[j])!=-1) and (any( key in str_i_list[k] for key in possibility_list )):
                df.at[i,column_add_list[j]] = column_add_list[j] + '?'   # -1表示”不确定“
#                 print(str_i_list[k])
            elif (str.find(str_i_list[k],column_add_list[j])!=-1) and (all(key not in str_i_list[k] for key in possibility_list)):
                df.at[i,column_add_list[j]] = column_add_list[j]   # 1表示”患该病“
#                 print(column_add_list[j])
            else:
                pass

In [ ]:
# # 多个子字符串
# key_list = ['？','待查','可能性大','待除外']
 
# # 字符串
# str ='我爱？,python待查,go,php'
 
# # if any(key in str for key in key_list):
# #     print('contains')
# for i in range(len(key_list)):
#     if key_list[i] in str:
#         print(key_list[i])

In [ ]:
# import difflib
# import string
# for i in range(df.shape[0]):
#     str_i = df.at[i,'阅片诊断']
#     str_tempt = re.split(r'双眼|右|左|；|及|，|、', str_i)
#     str_i_list = list(filter(None,str_tempt))    # list(filter(None,str)) :去掉re.split()产生的”空字符“项
#     print(str_i_list)
#     for j in range(len(column_add_list)):
#         for k in range(len(str_i_list)):
#             if (str.find(str_i_list[k],column_add_list[j])!=-1) and ('？' in str_i_list[k]):
#                 df.at[i,column_add_list[j]] = -1   # -1表示”不确定“
# #                 print(column_add_list[j])
#             elif (str.find(str_i_list[k],column_add_list[j])!=-1) and ('待查' in str_i_list[k]):
#                 df.at[i,column_add_list[j]] = -1   # -1表示”不确定“
# #                 print(column_add_list[j])
#             elif (str.find(str_i_list[k],column_add_list[j])!=-1) and ('可能性大' in str_i_list[k]):
#                 df.at[i,column_add_list[j]] = -1   # -1表示”不确定“
# #                 print(column_add_list[j])
#             elif(str.find(str_i_list[k],column_add_list[j])!=-1) and ('待除外'in str_i_list[k]):
#                 df.at[i,column_add_list[j]] = -1   # -1表示”不确定“
# #                 print(column_add_list[j])
#             elif(str.find(str_i_list[k],column_add_list[j])!=-1):
#                 df.at[i,column_add_list[j]] = 1   # 1表示”患该病“
# #                 print(column_add_list[j])
# #                 if "？" in str_i_list[k]:
#             else:
#                 pass


In [ ]:
df.columns

In [ ]:
df['色素变性'].value_counts()

In [ ]:
df.info()

In [ ]:
# ## 测试代码
# import difflib
# a = ['视网膜色素变性', '并发性白内障']
# for i in range(len(column_add_list)):
#     c = difflib.get_close_matches(column_add_list[i],a,1,cutoff=0.7)
#     if c:
#         print(c)
#         print(df.at[0,column_add_list[i]])
    

In [ ]:
# import string
# s =  '老年性白内障（皮质性）屈光不正双视网膜色素变性或无脉络膜症'
# name = '屈光'
# if str.find(s,name)!=-1:
#     print(name)

In [ ]:
# ## 测试代码
# import difflib
# a = ['老年性白内障（皮质性）屈光不正双视网膜色素变性或无脉络膜症']
# for i in range(len(column_add_list)):
#     c = difflib.get_close_matches(column_add_list[i],a,1,cutoff=0.2)
#     if c:
#         print(c)
#         print(column_add_list[i])

In [ ]:
# a = 'abcdefg'
# b = 'bcd'
# c = 'fgh'
# print('bcd|fgh' not in a)

In [ ]:

# # 多个子字符串
# key_list = ['java','python','go','c++']
 
# # 字符串
# str ='我爱java,python,go,php'
 
# for key in key_list:
#     if key in str:
#         print(key)

In [ ]:
# 多个子字符串
key_list = ['？','待查','可能性大']
 
# 字符串
str ='我爱,python,go,php'
 
if any(key in str for key in key_list):
    print('contains')


In [ ]:
# possibility_list = ['？|待查|可能性大']

In [ ]:
# import difflib
# for i in range(df.shape[0]):
#     str_i = df.at[i,'阅片诊断']
#     str_tempt = re.split(r'双眼|右|左|；|及|，|、', str_i)
#     str_i_list = list(filter(None,str_tempt))    # list(filter(None,str)) :去掉re.split()产生的”空字符“项
#     print(str_i_list)
#     for j in range(len(column_add_list)):
#         match_j = difflib.get_close_matches(column_add_list[j],str_i_list,1,cutoff=0.3)
#         if match_j and '？' in match_j[0]:   #Python判断一个字符串是否包含子串的几种方法,使用成员操作符 in
#             df.at[i,column_add_list[j]] = -1
#         elif match_j:
#             df.at[i,column_add_list[j]] = 1
#         else:
#             pass

In [ ]:
df[df['阅片诊断'].str.contains('待查')]['阅片诊断']

In [ ]:
a = '视锥视杆细胞营养不良？'
if '？' in a:
    print(1)
else:
    print(0)

In [ ]:
for i in range(df.shape[0]):
    if '？' in df.at[i,'脉络膜病变']:
        df.at[i,'脉络膜病变'] = '-1'
    else:
        df.at[i,'脉络膜病变'] = '0'

In [ ]:
df['阅片诊断'].value_counts()

In [ ]:
df[df['色素变性'].notnull()]['白内障']

In [ ]:
a = ['双结晶状视网膜色素变性？双眼底玻璃膜疣？双视网膜动脉硬化','双老年性白内障']

In [ ]:
re.findall((r'双眼*'),a)

In [ ]:
黄斑水肿 ——Null  锥杆营养不良——Null  玻璃膜疣——Null  血管堵塞  视网膜劈裂 

In [ ]:
for i in range(len(df.shape[0])):
    

In [ ]:
a = df.at[0,'图像描述']
a

# python按照多个字符对字符串进行分割的方法
这段python代码通过这规则表达式对字符串进行分割，使用\w作为分割符，只要不是字母和数字的就会被分割开来

import re

DATA = "Hey, you - what are you doing here! welcome to jb51?"

print re.findall(r"[\w']+", DATA)

# 在python 中split()使用多符号分割的例子
In [1]: import re
 
In [2]: words = '我，来。上海？吃？上海菜'
 
In [3]: wordlist = re.split('，|。|？',words)
 
In [4]: print(wordlist)
 
output：
['我', '来', '上海', '吃', '上海菜']

In [ ]:
b = re.findall(r"[\w]+",a)   # 这段python代码通过这规则表达式对字符串进行分割，使用\w作为分割符，只要不是字母和数字的就会被分割开来
b[0]

# Python 中re.split()方法
>>> import re
>>> line = 'aaa bbb ccc;ddd   eee,fff'
>>> line
'aaa bbb ccc;ddd   eee,fff'

(1)单字符切割
>>> re.split(r';',line)
['aaa bbb ccc', 'ddd\teee,fff']

(2)两个字符以上切割需要放在 [ ] 中
>>> re.split(r'[;,]',line)
['aaa bbb ccc', 'ddd\teee', 'fff']

(3)所有空白字符切割
>>> re.split(r'[;,\s]',line)    # \s	匹配任何空白字符，包括空格、制表符、换页符等等。等价于 [ \f\n\r\t\v]。注意 Unicode 正则表达式会匹配全角空格符。
['aaa', 'bbb', 'ccc', 'ddd', 'eee', 'fff']

(4)不想保留分隔符，以（?:...）的形式指定
>>> re.split(r'(?:[;])',line)
['aaa bbb ccc', 'ddd\teee,fff']

## Python3 三种办法解决split结果包含空字符串的问题
方法一：列表推导式

In [3]: [x for x in s.split(',') if x]
Out[3]: ['abc', 'fefdaf', '123fad', 'fsdfa']

方法二：调用filter库函数：

In [4]: list(filter(None,s.split(',')))
Out[4]: ['abc', 'fefdaf', '123fad', 'fsdfa']

方法三：使用正则表达式：

In [6]: re.findall('[a-z0-9]+',s)
Out[6]: ['abc', 'fefdaf', '123fad', 'fsdfa']

In [ ]:
b = a.split('，')
print(b)
b[0]

In [ ]:
b[1]

In [ ]:
m = re.match('.+视盘.*',a)
m

In [ ]:
b = re.split('，|,',a)
b[0]

In [ ]:
for i in range(len(b)):
    if  re.findall('.+视盘.+',b[i]):
        print(re.findall('.+视盘.+',b[i]))
    else:
        pass
    

# '(?P...)' 分组匹配——直接将匹配结果直接转为字典模式，方便使用

In [ ]:
res = re.search('(?P<视盘>\w{5})(?P<视网膜>\w{6})',a)

print(res.groupdict())

In [ ]:
import re
s = '1102231990xxxxxxxx'
res = re.search('(?P<province>\d{3})(?P<city>\d{3})(?P<born_year>\d{4})',s)
print(res.groupdict())

# re.split(pattern, string, [maxsplit], [flags])
pattern：表示模式字符串，由要匹配的正则表达式转换而来。

string：表示要匹配的字符串。

maxsplit：可选参数，表示最大的拆分次数。

flags：可选参数表示标志位，用于控制匹配方式，如是否区分子母大小写


In [ ]:
import re
 
pattern = r'[?|&]'           # 定义分隔符
url = 'http://www.baidu.com/login.jsp?username="wei"&pwd="123"' # 需要拆分的字符串
result = re.split(pattern, url) # 以pattern的值 分割字符串
print(result)